In [ ]:
!pip install pandas openpyxl dhlab

In [10]:
## CONFIG 

naob_file = "NAOB 2001-05.xlsx"
missing_file = "NAOB_konkordans/NAOB 2001-05 mangler.xlsx"


In [234]:
from typing import Iterable

## String formatting functions

def replace_non_alpha(string: str, strict: bool = False) -> str:
    """Replace non-alphanumeric punctuation marks with whitespace in the string."""
    if (string is None) or (not string) or (isinstance(string, float)):
        return None
    if isinstance(string, int):
        return str(string)
    if not string.isalnum():
        iterable_string = string.split() if strict else string

        for x in iterable_string: 
            string = string.replace(x, " ") if not x.isalnum() else string
        #print(string)
    return string.strip() 


def flip_names(author: str) -> str: 
    return " ".join(reversed(author.strip().split(" , ")))


def remove_author_annotation(author: str) -> str: 
    return author.replace(" (red.)", "").replace(" et al.", "")


def remove_initials(author: str) -> str:
    if author is None:
        return None
    return " ".join([x for x in author.split() if len(x) > 1])

def split_authors(authors: str, sep="/") -> Iterable[str]:
    if authors is None:
        return []
    return [auth.strip() for auth in authors.split(" " + sep + " ")]


def format_authors_for_comparison(authors: str) -> str: 
    """Format authors from dhlab.Corpus to compare with the NAOB data."""
    return [
        flip_names(auth).strip() for auth in split_authors(authors)
    ]


In [239]:
import pandas as pd
import  dhlab as dh
import dhlab.api.dhlab_api as api
import datetime
import re

# Les Excel-arket
df = pd.read_excel(naob_file, names=["Publikasjon", "Årstall"])


def parse_publication(pub_string):
    match = re.match(r'(.*?) – (.*?) \((.*?) (\d{4})\)', pub_string)
    if match:
        forfatter = match.group(1)
        full_tittel = match.group(2)
        sted = match.group(3)
        år = match.group(4)
        
        # Del opp tittel og undertittel ved siste forekomst av ': '
        tittel_deler = full_tittel.rsplit(': ', 1)
        tittel = tittel_deler[0].strip()
        undertittel = tittel_deler[1].strip() if len(tittel_deler) > 1 else None
        
        return {
            'Forfatter': forfatter,
            'Tittel': tittel,
            'Undertittel': undertittel,
            'Sted': sted,
            'År': år
        }
    return {}

# Anvend parsing-funksjonen på hver rad
df = pd.concat([df, df['Publikasjon'].apply(parse_publication).apply(pd.Series)], axis=1)

# Formater strengverdiene før spørring
#Fjern tegnsetting, del opp flere forfattere i liste, fjern initialer
query_df = df.copy()

# Fjern non-alphanumeriske tegn fra tittel
query_df["Tittel"] = df.Tittel.apply(replace_non_alpha)

# Fjern annotasjoner som angir flere forfattere/ roller
query_df["Forfatter"] = query_df.Forfatter.str.replace(" (red.)", "").replace(" et al.", "")

# Fjern non-alphanumeriske tegn fra forfatter
query_df["Forfatter"] = query_df.Forfatter.apply(replace_non_alpha)

# FJern initialer fra forfatter
query_df["Forfatter"] = query_df.Forfatter.apply(remove_initials)

# Del opp i navneliste
query_df["Forfatterliste"] = query_df.Forfatter.map(lambda x: split_authors(x, sep="og"))



def get_corpus_from_metadata(row):
    try:
        results = api.document_corpus(
            author=row['Forfatterliste'][0] if row['Forfatterliste'] else None,
            title=row['Tittel'],
            from_year = int(row['Årstall']),
            to_year = int(row['Årstall']) + 1,
            doctype = "digibok",
        )
    except Exception as e:
        print(f"Failed to query metadata for row {row.name}, {row['Tittel']} by {row['Forfatter']}: {e}")
        #raise e
        results = pd.DataFrame()
    return results


for idx, row in df.iterrows():
    query_row = query_df.loc[idx]
    metadata = get_corpus_from_metadata(query_row)

    if not metadata.empty:
        #urn = find_best_match(row, metadata)
        urn = metadata.iloc[0].get('urn', '')
        df.loc[idx, 'URN'] = urn
        df.loc[idx, 'URL'] = 'https://urn.nb.no/' + df.loc[idx, 'URN']
        
#df.URL = df.URN.map(lambda x: f"https://urn.nb.no/{x}" if x else None)

# Lagre resultatet til en ny Excel-fil

today = datetime.datetime.today().strftime("%Y-%m-%d")

df.to_excel(f'NAOB_2001-2005_med_URN_{today}.xlsx', index=False)

In [249]:
df.to_excel(f'NAOB_2001-2005_med_URN_{today}.xlsx', index=False)


In [251]:
df.loc[df.Forfatter.isna()]

,Publikasjon,Årstall,Forfatter,Tittel,Undertittel,Sted,År,URN,URL
107,Festbok til Olav Angell i anledning av 70-årsd...,2002,NaN,NaN,NaN,NaN,NaN,URN:NBN:no-nb_digibok_2016021807550,https://urn.nb.no/URN:NBN:no-nb_digibok_201602...
154,"Naturgressboka : bygging, drift, vedlikehold a...",2002,NaN,NaN,NaN,NaN,NaN,URN:NBN:no-nb_digibok_2014110407587,https://urn.nb.no/URN:NBN:no-nb_digibok_201411...
235,Metallurgisk ordbok (Oslo 2003) 2. utg. (1. ut...,2003,NaN,NaN,NaN,NaN,NaN,URN:NBN:no-nb_digibok_2020040748007,https://urn.nb.no/URN:NBN:no-nb_digibok_202004...
371,Grovvitser (Oslo 2005) utgitt som bilag til Vi...,2005,NaN,NaN,NaN,NaN,NaN,URN:NBN:no-nb_digibok_2011111705061,https://urn.nb.no/URN:NBN:no-nb_digibok_201111...


In [250]:
df.loc[df.Forfatter.isna()].URL.to_list()

['https://urn.nb.no/URN:NBN:no-nb_digibok_2016021807550',
 'https://urn.nb.no/URN:NBN:no-nb_digibok_2014110407587',
 'https://urn.nb.no/URN:NBN:no-nb_digibok_2020040748007',
 'https://urn.nb.no/URN:NBN:no-nb_digibok_2011111705061']

In [ ]:


# Funksjon for å finne beste treff
def find_best_match(row, metadata):
    
    author=row['Forfatter']
    title=row['Tittel']
    year = int(row['Årstall'])
    query
    
#    search_string = row['Søkestreng'].lower()
    best_match = None
    best_score = 0
    
    for idx, item in metadata.iterrows():
        dh_title = item.get('title', '').lower()
        authors = ' '.join(item.get('authors', [])).lower()
        year = str(item.get('year', ''))
        
        #item_string = f"{title} {authors} {year}"
        
        # Enkel poengberegning basert på hvor mange ord som matcher
        score = sum(1 for word in search_string.split() if word in item_string)
        
        if score > best_score:
            best_score = score
            best_match = item
    
    return best_match['urn'] if best_match else ''




In [194]:

for idx, row in df.iterrows():
    query_row = query_df.loc[idx]
    metadata = get_corpus_from_metadata(query_row)

    if not metadata.empty:
        #urn = find_best_match(row, metadata)
        urn = metadata.iloc[0].get('urn', '')
        df.loc[idx, 'URN'] = urn
        df.loc[idx, 'URL'] = 'https://urn.nb.no/' + df.loc[idx, 'URN']
        
#df.URL = df.URN.map(lambda x: f"https://urn.nb.no/{x}" if x else None)

# Lagre resultatet til en ny Excel-fil
#df.to_excel('NAOB_2001-2005_med_URN.xlsx', index=False)


Failed to query metadata for row 26, Frihetens bitre tiår by Hans Wilhelm Steinfeld: Expecting value: line 1 column 1 (char 0)
Failed to query metadata for row 56, Kometenes øyeblikk by Odd W  Surén: Expecting value: line 1 column 1 (char 0)
Failed to query metadata for row 63, Rød engel by Sissel Lange Nielsen: Expecting value: line 1 column 1 (char 0)
Failed to query metadata for row 87, Dr  Zellwegers gave by Anne B  Ragde: Expecting value: line 1 column 1 (char 0)
Failed to query metadata for row 100, Ayatollah highway by Christopher Friis Baastad Grøndahl og Arne Svingen: Expecting value: line 1 column 1 (char 0)
Failed to query metadata for row 109, Moderne norsk by Finn Erik Vinje: Expecting value: line 1 column 1 (char 0)
Failed to query metadata for row 114, Alexander L  Kielland    I slekt med hele byen by Gunnar A  Skadberg: Expecting value: line 1 column 1 (char 0)
Failed to query metadata for row 132, Mannen som gikk på jorden by Ketil Bjørnstad  Torstein Bieler og Lage Fo

In [192]:
df.iloc[26]

Publikasjon    Hans-Wilhelm Steinfeld – Frihetens bitre tiår ...
Årstall                                                     2001
Forfatter                                 Hans-Wilhelm Steinfeld
Tittel                                      Frihetens bitre tiår
Undertittel                                                 None
Sted                                                        Oslo
År                                                          2001
URN                                                          NaN
URL                                                          NaN
Name: 26, dtype: object

In [132]:
## Test parse titler 
examples = [
    "Forfatter – Hovedtittel: Undertittel (Oslo 2001)",
    "Forfatter1 og Forfatter2 – Hovedtittel: Undertittel (Oslo 2001)",
    "Forfatter – Kompleks tittel: med flere: kolon: Faktisk undertittel (Oslo 2001)",
    "Forfatter – Tittel uten undertittel (Oslo 2001)"
]

for example in examples:
    result = parse_publication(example)
    print(f"Original: {example}")
    print(f"Parsed: {result}\n")


Original: Forfatter – Hovedtittel: Undertittel (Oslo 2001)
Parsed: {'Forfatter': 'Forfatter', 'Tittel': 'Hovedtittel', 'Undertittel': 'Undertittel', 'Sted': 'Oslo', 'År': '2001'}

Original: Forfatter1 og Forfatter2 – Hovedtittel: Undertittel (Oslo 2001)
Parsed: {'Forfatter': 'Forfatter1 og Forfatter2', 'Tittel': 'Hovedtittel', 'Undertittel': 'Undertittel', 'Sted': 'Oslo', 'År': '2001'}

Original: Forfatter – Kompleks tittel: med flere: kolon: Faktisk undertittel (Oslo 2001)
Parsed: {'Forfatter': 'Forfatter', 'Tittel': 'Kompleks tittel: med flere: kolon', 'Undertittel': 'Faktisk undertittel', 'Sted': 'Oslo', 'År': '2001'}

Original: Forfatter – Tittel uten undertittel (Oslo 2001)
Parsed: {'Forfatter': 'Forfatter', 'Tittel': 'Tittel uten undertittel', 'Undertittel': None, 'Sted': 'Oslo', 'År': '2001'}



In [170]:
c = dh.Corpus(author= "Anne Enger", title="Grønn dame rød klut")


In [173]:
df.iloc[0].Tittel



'Grønn dame, rød klut'

In [141]:
#c = dh.Corpus(author= "Ibsen")
c.frame.authors.map(format_authors_for_comparison).to_list()

[['Henrik Ibsen', 'Steinar Vestli'],
 ['Henrik Ibsen', 'Nils Lund'],
 ['Henrik Ibsen'],
 ['Henrik Ibsen', 'Suzannah Ibsen'],
 ['Sigurd Ibsen'],
 ['Hilde Fiskum',
  'Gunn Bakke Hansen',
  'Mari Dahl Sæther',
  'Mathilde Alfrida Jakobsen Skarpsno',
  'Stian Isaksen',
  'Ola Otnes',
  'Henrik Ibsen'],
 ['Henrik Ibsen', 'Sverre Mo . Skrede'],
 ['Henrik Ibsen'],
 ['Henrik Ibsen', 'Alexander Mørk-Eidem'],
 ['Henrik Ibsen']]

In [138]:

metadata = corpus.frame 


# Funksjon for å finne beste treff
def find_best_match(row, metadata):
    search_string = row['Søkestreng'].lower()
    best_match = None
    best_score = 0
    
    for item in metadata:
        title = item.get('title', '').lower()
        authors = ' '.join(item.get('authors', [])).lower()
        year = str(item.get('year', ''))
        
        item_string = f"{title} {authors} {year}"
        
        # Enkel poengberegning basert på hvor mange ord som matcher
        score = sum(1 for word in search_string.split() if word in item_string)
        
        if score > best_score:
            best_score = score
            best_match = item
    
    return best_match['urn'] if best_match else ''

# Finn URN for hver publikasjon
df['URN'] = df.apply(lambda row: find_best_match(row, metadata), axis=1)

# Lag fullstendig URL
df['URL'] = 'https://urn.nb.no/' + df['URN']

# Lagre resultatet til en ny Excel-fil
df.to_excel('publikasjoner_med_url.xlsx', index=False)


NameError: name 'corpus' is not defined